# neuralcoref 사용법

In [1]:
import spacy
nlp = spacy.load('en')

import neuralcoref
neuralcoref.add_to_pipe(nlp)

doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)
print(doc._.coref_clusters)

True
[My sister: [My sister, She], a dog: [a dog, him]]


In [2]:
import spacy
nlp = spacy.load('en')

import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)
print(doc._.coref_clusters)

True
[My sister: [My sister, She], a dog: [a dog, him]]


In [3]:
print(f'doc은\n{doc}')
print(doc._.coref_resolved)
print(doc._.coref_scores)
print('')
span = doc[-7:-5]
print(f'span은\n{span}')

print(span._.is_coref)
print(span._.coref_cluster)
print(span._.coref_scores)
print('')
token = doc[-10]
print(f'token은\n{token}')

print(token._.in_coref)
print(token._.coref_clusters)

doc은
My sister has a dog. She loves him.
My sister has a dog. My sister loves a dog.
{My sister: {My sister: 1.3110305070877075}, a dog: {a dog: 1.804752230644226, My sister: -1.6715972423553467}, She: {She: -0.10834205150604248, My sister: 8.058426856994629, a dog: -1.0625176429748535}, him: {him: -1.870743989944458, My sister: 3.1147186756134033, a dog: 4.356405258178711, She: -3.1379528045654297}}

span은
a dog
True
a dog: [a dog, him]
{a dog: 1.804752230644226, My sister: -1.6715972423553467}

token은
My
True
[My sister: [My sister, She]]


In [4]:
import spacy
import neuralcoref

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)
doc = nlp(u'Deepika has a dog. She loves him. The movie star has always been fond of animals')
print(doc._.coref_clusters)
print(doc._.coref_resolved)

[Deepika: [Deepika, She, him, The movie star]]
Deepika has a dog. Deepika loves Deepika. Deepika has always been fond of animals


In [5]:
nlp.remove_pipe("neuralcoref")
neuralcoref.add_to_pipe(nlp, conv_dict={'Deepika':['woman','actress']})

doc = nlp(u'Deepika has a dog. She loves him. The movies star has always been fond of animals')
doc._.coref_clusters

[Deepika: [Deepika, She, The movies star], a dog: [a dog, him]]

In [6]:
doc._.coref_resolved

'Deepika has a dog. Deepika loves a dog. Deepika has always been fond of animals'

# Coreference Resolution

In [7]:
# spacy load하기
nlp = spacy.load('en')

# spacy pipe에 neural coref 추가하기
neuralcoref.add_to_pipe(nlp)

def coref_resolution(text):
    '''
    주어진 텍스트에 대해 coreference resolution 수행
    '''
    doc = nlp(text)
    return doc._.coref_resolved

text = '''
Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.
'''

print(text)
print('------------------------------------------')
print(coref_resolution(text))
    


Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.

------------------------------

# Named Entity Linking

In [9]:
import spacy

nlp = spacy.load("en")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [11]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)